In [103]:
import pandas as pd
import numpy as np


In [104]:
df = pd.read_csv('merged.csv')

In [105]:
df

,Unnamed: 0,created_at_month,patient_id,folk_1_code,folk_2_code,folk_3_code,folk_4_code,comment_len,service_code,service_level_3_category_hash,...,standartized_diagnosis_len,diagnosis_uuid,diagnosis_category_level_3_hash,service_status,diagnosis_category_level_1_hash,service_quantity,cnt_visit,дата убытка,loss,month
0,0,2020-07,3383,1,1,1,1,211.000000,1,1,...,0.0,1,1,0.428571,1,1.000,1,2020-07-19,6.840,2020-07
1,1,2020-07,3533,2,1,2,1,72.166667,2,2,...,0.0,1,1,0.000000,1,3.125,2,2020-07-31,2.090,2020-07
2,2,2020-07,3533,2,1,2,1,72.166667,2,2,...,0.0,1,1,0.000000,1,3.125,2,2020-07-19,3.150,2020-07
3,3,2020-07,3533,2,1,2,1,72.166667,2,2,...,0.0,1,1,0.000000,1,3.125,2,2020-07-17,9.200,2020-07
4,4,2020-07,3673,1,1,1,1,31.000000,1,1,...,0.0,1,1,0.000000,1,5.000,1,2020-07-15,2.720,2020-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226374,226374,2022-02,203973,1,1,1,1,117.000000,1,1,...,52.0,1,1,0.000000,1,1.000,1,2022-02-17,1.700,2022-02
226375,226375,2022-02,203992,1,1,1,1,356.000000,1,1,...,46.0,1,1,0.000000,1,1.000,1,2022-02-25,0.702,2022-02
226376,226376,2022-02,203999,1,1,1,1,30.500000,1,1,...,70.0,1,1,0.000000,1,1.000,2,2022-02-24,6.050,2022-02
226377,226377,2022-02,203999,1,1,1,1,30.500000,1,1,...,70.0,1,1,0.000000,1,1.000,2,2022-02-25,1.650,2022-02


In [106]:
df.columns

Index(['Unnamed: 0', 'created_at_month', 'patient_id', 'folk_1_code',
       'folk_2_code', 'folk_3_code', 'folk_4_code', 'comment_len',
       'service_code', 'service_level_3_category_hash',
       'service_level_2_category_hash', 'service_level', 'status',
       'is_paid_by_patient', 'has_doctor', 'doctor_len', 'diagnosis_len',
       'diagnosis_clarification_len', 'standartized_diagnosis_len',
       'diagnosis_uuid', 'diagnosis_category_level_3_hash', 'service_status',
       'diagnosis_category_level_1_hash', 'service_quantity', 'cnt_visit',
       'дата убытка', 'loss', 'month'],
      dtype='object')

In [107]:
df.loc[df['loss'].isna(), 'loss'] = df['loss'].mean()

In [108]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [109]:
df.isna().sum()

created_at_month                   0
patient_id                         0
folk_1_code                        0
folk_2_code                        0
folk_3_code                        0
folk_4_code                        0
comment_len                        0
service_code                       0
service_level_3_category_hash      0
service_level_2_category_hash      0
service_level                      0
status                             0
is_paid_by_patient                 0
has_doctor                         0
doctor_len                         0
diagnosis_len                      0
diagnosis_clarification_len        0
standartized_diagnosis_len         0
diagnosis_uuid                     0
diagnosis_category_level_3_hash    0
service_status                     0
diagnosis_category_level_1_hash    0
service_quantity                   0
cnt_visit                          0
дата убытка                        0
loss                               0
month                              0
d

In [110]:
df.dtypes

created_at_month                    object
patient_id                           int64
folk_1_code                          int64
folk_2_code                          int64
folk_3_code                          int64
folk_4_code                          int64
comment_len                        float64
service_code                         int64
service_level_3_category_hash        int64
service_level_2_category_hash        int64
service_level                      float64
status                              object
is_paid_by_patient                    bool
has_doctor                            bool
doctor_len                           int64
diagnosis_len                        int64
diagnosis_clarification_len          int64
standartized_diagnosis_len         float64
diagnosis_uuid                       int64
diagnosis_category_level_3_hash      int64
service_status                     float64
diagnosis_category_level_1_hash      int64
service_quantity                   float64
cnt_visit  

In [111]:
df['created_at_month'] = pd.to_datetime(df['created_at_month'], format='%Y-%m')
df['дата убытка'] = pd.to_datetime(df['дата убытка'], format='%Y-%m-%d')

df['created_month'] = df['created_at_month'].dt.month
df['created_year'] = df['created_at_month'].dt.year


df['loss_month'] = df['дата убытка'].dt.month
df['loss_year'] = df['дата убытка'].dt.year
df['loss_day'] = df['дата убытка'].dt.day

In [112]:
#К статусам применим label-encoding
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
lb.fit(df['status'])
df['status'] = lb.transform(df['status'])


In [113]:
from sklearn.model_selection import train_test_split

X, y = df.drop(columns=['loss', 'created_at_month', 'дата убытка', 'month']), df['loss']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def scorer_abs(estimator, X, y):
    return mean_absolute_error(y, estimator.predict(X))

def scorer_rmse(estimator, X, y):
    return mean_squared_error(y, estimator.predict(X), squared=False)

param_grid = {'metric' : ('cosine', 'minkowski', 'manhattan', 'euclidean'),
              'p' : [1, 2],
              'n_neighbors' : np.logspace(0.4, 2, 20).astype(int)
             }

knn = KNeighborsRegressor()
clf = GridSearchCV(knn, param_grid, scoring=scorer_rmse, n_jobs=-1)
clf.fit(X_train, y_train)

print('Best score ', clf.best_score_)
print('Best params ', clf.best_params_)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print("Test RMSE = %.4f" % mean_squared_error(y_test, y_pred, squared=False))
print("Test MAE = %.4f" % mean_absolute_error(y_test, y_pred))